In [1]:
from datetime import datetime
import pytz
import Bio

In [2]:
import os,sys
import pandas as pd

pd.set_option('display.max_columns', 20) # 최대 20개의 컬럼만 표시
pd.set_option('display.expand_frame_repr', False) # 넓은 DataFrame이 화면 너비를 초과하더라도 여러 줄로 wrapping 되지 않고, 가능한 한 한 줄에 모든 컬럼

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from timeit import default_timer as timer
import re
import matplotlib 
import matplotlib.pyplot as plt

In [4]:
valid_df = pd.read_csv(f'/home/bio_science/structure_prediction/rna_folding_kaggle/data/validation_sequences.csv')
label_df = pd.read_csv('/home/bio_science/structure_prediction/rna_folding_kaggle/data/validation_labels.csv')

In [5]:
label_df.head()

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,z_37,x_38,y_38,z_38,x_39,y_39,z_39,x_40,y_40,z_40
0,R1107_1,G,1,-5.499,8.520000,8.605000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
1,R1107_2,G,2,-5.826,10.453000,14.010000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2,R1107_3,G,3,-5.849,14.768000,17.584999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
3,R1107_4,G,4,-5.784,19.985001,18.666000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
4,R1107_5,G,5,-5.755,25.533001,17.132999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18


In [6]:
label_df['target_id'] = label_df['ID'].apply(lambda x: '_'.join(x.split('_')[:-1]))

In [7]:
label_df.head()

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,x_38,y_38,z_38,x_39,y_39,z_39,x_40,y_40,z_40,target_id
0,R1107_1,G,1,-5.499,8.520000,8.605000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107
1,R1107_2,G,2,-5.826,10.453000,14.010000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107
2,R1107_3,G,3,-5.849,14.768000,17.584999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107
3,R1107_4,G,4,-5.784,19.985001,18.666000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107
4,R1107_5,G,5,-5.755,25.533001,17.132999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,R1107


In [8]:
print('len(valid_df)',len(valid_df))
print(valid_df.iloc[0])
print('')

len(valid_df) 12
target_id                                                      R1107
sequence           GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUU...
temporal_cutoff                                           2022-05-28
description        CPEB3 ribozyme\nHuman\nhuman CPEB3 HDV-like ri...
all_sequences      >7QR4_1|Chain A|U1 small nuclear ribonucleopro...
Name: 0, dtype: object



In [10]:
NUM_CONF=5
MAX_LENGTH=480
DEVICE='cuda'

In [12]:
from collections import defaultdict

In [13]:
tokens=defaultdict(lambda: 4)

In [15]:
tokens['A'] = 0

In [18]:
tokens['B'] = 1

In [19]:
tokens['A'], tokens['B']

(0, 1)

In [2]:
import pandas as pd

In [3]:
train_sequences = pd.read_csv('/home/bio_science/structure_prediction/rna_folding_kaggle/data/train_sequences.v2.csv')

In [5]:
train_labels = pd.read_csv('/home/bio_science/structure_prediction/rna_folding_kaggle/data/train_labels.v2.csv')

In [6]:
train_labels.head()

,ID,resname,resid,x_1,y_1,z_1
0,7TAX_M_1,C,1,187.126007,148.246002,210.417999
1,7TAX_M_2,U,2,185.255997,152.968002,204.617996
2,7TAX_M_3,A,3,189.360992,161.802002,205.214996
3,7TAX_M_4,A,4,186.000000,156.595993,209.951996
4,7TAX_M_5,G,5,181.947998,158.186996,213.610992


In [8]:
train_labels['pdb_id'] = train_labels['ID'].apply(lambda x : x.split('_')[0] + '_' + x.split('_')[1])

In [9]:
train_labels.head()

,ID,resname,resid,x_1,y_1,z_1,pdb_id
0,7TAX_M_1,C,1,187.126007,148.246002,210.417999,7TAX_M
1,7TAX_M_2,U,2,185.255997,152.968002,204.617996,7TAX_M
2,7TAX_M_3,A,3,189.360992,161.802002,205.214996,7TAX_M
3,7TAX_M_4,A,4,186.000000,156.595993,209.951996,7TAX_M
4,7TAX_M_5,G,5,181.947998,158.186996,213.610992,7TAX_M


In [11]:
train_sequences.head()

,target_id,sequence,temporal_cutoff,description,all_sequences
0,7TAX_M,CUAAGAAAUUCACGGCGGGCUUGAUGUCCGCGUCUACCUGAUUCAC...,2022-09-21,Cryo-EM structure of the Csy-AcrIF24-promoter ...,>7TAX_1|Chain A|CRISPR-associated protein Csy1...
1,4WF1_CA,AAUUGAAGAGUUUGAUCAUGGCUCAGAUUGAACGCUGGCGGCAGGC...,2014-11-05,Crystal structure of the E. coli ribosome boun...,">4WF1_1|Chains A[auth AA], BB[auth CA]|16S rRN..."
2,8PVA_b,UGCCUGGCGGCCGUAGCGCGGUGGUCCCACCUGACCCCAUGCCGAA...,2023-11-29,Structure of bacterial ribosome determined by ...,>8PVA_1|Chain A|16S rRNA|Escherichia coli (562...
3,8OVE_BB,CAACUGCAGACCGUACUCAUCACCGCAUCAGGUCCCCAAGCAUCGA...,2023-11-29,CRYO-EM STRUCTURE OF TRYPANOSOMA BRUCEI PROCYC...,>8OVE_1|Chain A[auth AA]|SSU rRNA|Trypanosoma ...
4,8JDL_w,UACCUGGUUGAUCCUGCCAGUAGCAUUGCUUGCCAAAGAUUAAGCC...,2023-12-06,Structure of the Human cytoplasmic Ribosome wi...,>8JDL_1|Chain A|mRNA|Homo sapiens (9606)\nUUAU...


In [10]:
train_sequences['target_id'].head()

0     7TAX_M
1    4WF1_CA
2     8PVA_b
3    8OVE_BB
4     8JDL_w
Name: target_id, dtype: object

In [18]:
all_xyz = []
for pdb_id in train_sequences['target_id']:
    df = train_labels[train_labels['pdb_id'] == pdb_id]

    print(df)

    xyz = df[['x_1', 'y_1', 'z_1']].to_numpy().astype('float32')
    print(xyz, type(xyz), xyz.shape)

    print(xyz<1e-17, type(xyz<1e-17), (xyz<1e-17).shape)
    xyz[xyz<1e-17] = float('NaN')
    all_xyz.append(xyz)
    break

           ID resname  resid         x_1         y_1         z_1  pdb_id
0    7TAX_M_1       C      1  187.126007  148.246002  210.417999  7TAX_M
1    7TAX_M_2       U      2  185.255997  152.968002  204.617996  7TAX_M
2    7TAX_M_3       A      3  189.360992  161.802002  205.214996  7TAX_M
3    7TAX_M_4       A      4  186.000000  156.595993  209.951996  7TAX_M
4    7TAX_M_5       G      5  181.947998  158.186996  213.610992  7TAX_M
..        ...     ...    ...         ...         ...         ...     ...
56  7TAX_M_57       G     57  132.528000  109.032997  176.059006  7TAX_M
57  7TAX_M_58       C     58  130.798004  108.698997  170.755005  7TAX_M
58  7TAX_M_59       A     59  129.899002  110.066002  165.324005  7TAX_M
59  7TAX_M_60       G     60  132.529999  111.850998  159.748993  7TAX_M
60  7TAX_M_61       C     61         NaN         NaN         NaN  7TAX_M

[61 rows x 7 columns]
[[187.126 148.246 210.418]
 [185.256 152.968 204.618]
 [189.361 161.802 205.215]
 [186.    156.596 20

In [19]:
config = {
    "seed": 0,
    "cutoff_date": "2020-01-01",
    "test_cutoff_date": "2022-05-01",
    "max_len": 256,
    "batch_size": 1,
    "learning_rate": 1e-4,
    "weight_decay": 0.0,
    "mixed_precision": "bf16",
    "model_config_path": "../working/configs/pairwise.yaml",  # Adjust path as needed
    "epochs": 1,
    "cos_epoch": 0,
    "loss_power_scale": 1.0,
    "max_cycles": 1,
    "grad_clip": 0.1,
    "gradient_accumulation_steps": 1,
    "d_clamp": 30,
    "max_len_filter": 9999999,
    "min_len_filter":10,
    "structural_violation_epoch": 50,
    "balance_weight": False,
    "n_times": 1000,
}

In [23]:
import numpy as np

In [31]:
filter_nan = []
max_len = 0
for xyz in all_xyz:
    print(xyz, type(xyz))
    if len(xyz) > max_len:
        max_len = len(xyz)

    filter_nan.append((np.isnan(xyz).mean() <= 0.5) & \
    (len(xyz)<config['max_len_filter']) & \
    (len(xyz)>config['min_len_filter']) )


[[187.126 148.246 210.418]
 [185.256 152.968 204.618]
 [189.361 161.802 205.215]
 [186.    156.596 209.952]
 [181.948 158.187 213.611]
 [177.646 163.181 214.16 ]
 [173.88  167.345 211.883]
 [166.337 171.126 210.593]
 [168.327 176.514 206.157]
 [163.816 179.339 207.056]
 [160.164 182.846 207.755]
 [157.792 186.915 204.088]
 [157.158 189.005 198.698]
 [152.66  192.781 192.645]
 [157.734 192.806 187.802]
 [154.684 196.122 184.126]
 [153.8   200.252 181.389]
 [154.312 201.403 175.669]
 [155.549 199.982 170.549]
 [154.211 199.162 162.055]
 [160.19  195.037 161.332]
 [159.291 196.51  156.358]
 [161.04  197.188 150.417]
 [162.789 193.873 145.997]
 [163.77  188.684 143.856]
 [164.069 183.063 137.584]
 [166.558 177.619 141.263]
 [168.46  174.762 137.156]
 [170.408 172.246 133.37 ]
 [171.267 166.194 132.894]
 [170.009 160.922 134.319]
 [168.633 152.67  132.593]
 [167.908 151.195 139.828]
 [168.569 146.26  138.383]
 [168.769 140.641 138.849]
 [166.788 135.88  141.479]
 [162.474 133.513 145.156]
 

In [26]:
filter_nan

[True]

In [32]:
filter_nan = np.array(filter_nan)

In [33]:
filter_nan

array([ True])

In [34]:
non_nan_indices = np.arange(len(filter_nan))[filter_nan]

In [36]:
non_nan_indices

array([0])

In [37]:
train_sequences = train_sequences.loc[non_nan_indices]

In [39]:
train_sequences = train_sequences.reset_index(drop=True)

In [40]:
train_sequences

,target_id,sequence,temporal_cutoff,description,all_sequences
0,7TAX_M,CUAAGAAAUUCACGGCGGGCUUGAUGUCCGCGUCUACCUGAUUCAC...,2022-09-21,Cryo-EM structure of the Csy-AcrIF24-promoter ...,>7TAX_1|Chain A|CRISPR-associated protein Csy1...


In [42]:
all_xyz=[all_xyz[i] for i in non_nan_indices]

In [43]:
all_xyz

[array([[187.126, 148.246, 210.418],
        [185.256, 152.968, 204.618],
        [189.361, 161.802, 205.215],
        [186.   , 156.596, 209.952],
        [181.948, 158.187, 213.611],
        [177.646, 163.181, 214.16 ],
        [173.88 , 167.345, 211.883],
        [166.337, 171.126, 210.593],
        [168.327, 176.514, 206.157],
        [163.816, 179.339, 207.056],
        [160.164, 182.846, 207.755],
        [157.792, 186.915, 204.088],
        [157.158, 189.005, 198.698],
        [152.66 , 192.781, 192.645],
        [157.734, 192.806, 187.802],
        [154.684, 196.122, 184.126],
        [153.8  , 200.252, 181.389],
        [154.312, 201.403, 175.669],
        [155.549, 199.982, 170.549],
        [154.211, 199.162, 162.055],
        [160.19 , 195.037, 161.332],
        [159.291, 196.51 , 156.358],
        [161.04 , 197.188, 150.417],
        [162.789, 193.873, 145.997],
        [163.77 , 188.684, 143.856],
        [164.069, 183.063, 137.584],
        [166.558, 177.619, 141.263],
 

In [44]:
data = {
    "sequence": train_sequences['sequence'].to_list(),
    "temporal_cutoff": train_sequences['temporal_cutoff'].to_list(),
    "description": train_sequences['description'].to_list(),
    "all_sequences": train_sequences['all_sequences'].to_list(),
    "xyz": all_xyz # C1' 좌표 리스트
}

In [45]:
data

{'sequence': ['CUAAGAAAUUCACGGCGGGCUUGAUGUCCGCGUCUACCUGAUUCACUGCCGUAUAGGCAGC'],
 'temporal_cutoff': ['2022-09-21'],
 'description': ['Cryo-EM structure of the Csy-AcrIF24-promoter DNA complex'],
 'all_sequences': [">7TAX_1|Chain A|CRISPR-associated protein Csy1|Pseudomonas (286)\nMTSPLPTPTWQELRQFIESFIQERLQGKLDKLQPDEDDKRQTLLATHRREAWLADAARRVGQLQLVTHTLKPIHPDARGSNLHSLPQAPGQPGLAGSHELGDRLVSDVVGNAAALDVFKFLSLQYQGKNLLNWLTEDSAEALQALSDNAEQAREWRQAFIGITTVKGAPASHSLAKQLYFPLPGSGYHLLAPLFPTSLVHHVHALLREARFGDAAKAAREARSRQESWPHGFSEYPNLAIQKFGGTKPQNISQLNNERRGENWLLPSLPPNWQRQNVNAPMRHSSVFEHDFGRTPEVSRLTRTLQRFLAKTVHNNLAIRQRRAQLVAQICDEALQYAARLRELEPGWSATPGCQLHDAEQLWLDPLRAQTDETFLQRRLRGDWPAEVGNRFANWLNRAVSSDSQILGSPEAAQWSQELSKELTMFKEILEDERD\n>7TAX_2|Chain B|CRISPR type I-F/YPEST-associated protein Csy2|Pseudomonas (286)\nMSVTDPEALLLLPRLSIQNANAISSPLTWGFPSPGAFTGFVHALQRRVGISLDIELDGVGIVCHRFEAQISQPAGKRTKVFNLTRNPLNRDGSTAAIVEEGRAHLEVSLLLGVHGDGLDDHPAQEIARQVQEQAGAMRLAGGSILPWCNERFPAPNAELLMLGGSDEQRRKNQRRLTRRLLPGFALVSREALLQQHLETLRT

In [46]:
all_index = np.arange(len(data['sequence']))

In [47]:
all_index

array([0])

In [48]:
len(data['sequence'])

1

In [49]:
cutoff_date = pd.Timestamp(config['cutoff_date'])

In [50]:
test_cutoff_date = pd.Timestamp(config['test_cutoff_date'])

In [56]:
cutoff_date

Timestamp('2020-01-01 00:00:00')

In [57]:
data['temporal_cutoff']

['2022-09-21']

In [58]:
test_cutoff_date

Timestamp('2022-05-01 00:00:00')

In [ ]:
train_index = [i for i, d in enumerate(data['temporal_cutoff']) if pd.Timestamp(d) <= cutoff_date]
test_index = [i for i, d in enumerate(data['temporal_cutoff']) if pd.Timestamp(d) > cutoff_date and pd.Timestamp(d) <= test_cutoff_date]

In [53]:
train_index, test_index

([], [])

In [55]:
print(f"Train size: {len(train_index)}")
print(f"Test size: {len(test_index)}")

Train size: 0
Test size: 0


In [59]:
from tqdm import tqdm

In [61]:
train_sequences = pd.read_csv('/home/bio_science/structure_prediction/rna_folding_kaggle/data/train_sequences.v2.csv')
train_labels = pd.read_csv('/home/bio_science/structure_prediction/rna_folding_kaggle/data/train_labels.v2.csv')

train_labels["pdb_id"] = train_labels["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])

all_xyz=[]

for pdb_id in tqdm(train_sequences['target_id']):
    df = train_labels[train_labels["pdb_id"]==pdb_id] # 현재 시퀀스에 해당하는 좌표만 필터링
    #break
    xyz=df[['x_1','y_1','z_1']].to_numpy().astype('float32') # C1' 좌표만 추출 
    xyz[xyz<-1e17]=float('Nan') # 큰 음수 값은 NaN으로 처리 (마스킹)
    all_xyz.append(xyz)

# filter the data
# Filter and process data
filter_nan = []
max_len = 0
for xyz in all_xyz:
    if len(xyz) > max_len:
        max_len = len(xyz)
    #fill -1e18 masked sequences to nans

    #sugar_xyz = np.stack([nt_xyz['sugar_ring'] for nt_xyz in xyz], axis=0)
    # NaN 비율 50% 이하
    # 너무 긴 시퀀스 제외
    # 너무 짧은 시퀀스 제외
    filter_nan.append((np.isnan(xyz).mean() <= 0.5) & \
                    (len(xyz)<config['max_len_filter']) & \
                    (len(xyz)>config['min_len_filter']))

print(f"Longest sequence in train: {max_len}")

filter_nan = np.array(filter_nan)
non_nan_indices = np.arange(len(filter_nan))[filter_nan] # 조건을 만족하는 인덱스만 추출

train_sequences = train_sequences.loc[non_nan_indices].reset_index(drop=True)
all_xyz=[all_xyz[i] for i in non_nan_indices]

data={
    "sequence":train_sequences['sequence'].to_list(), # 문자열 시퀀스
    "temporal_cutoff": train_sequences['temporal_cutoff'].to_list(),
    "description": train_sequences['description'].to_list(),
    "all_sequences": train_sequences['all_sequences'].to_list(),
    "xyz": all_xyz # C1' 좌표 리스트
}

# Split data into train and test
all_index = np.arange(len(data['sequence']))
cutoff_date = pd.Timestamp(config['cutoff_date'])
test_cutoff_date = pd.Timestamp(config['test_cutoff_date'])
train_index = [i for i, d in enumerate(data['temporal_cutoff']) if pd.Timestamp(d) <= cutoff_date]
test_index = [i for i, d in enumerate(data['temporal_cutoff']) if pd.Timestamp(d) > cutoff_date and pd.Timestamp(d) <= test_cutoff_date]

print(f"Train size: {len(train_index)}")
print(f"Test size: {len(test_index)}")


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5135/5135 [17:14<00:00,  4.97it/s]


Longest sequence in train: 4417
Train size: 2507
Test size: 801


In [63]:
import torch

In [65]:
xyz = torch.tensor(np.array(data['xyz'][0]), dtype=torch.float32)

In [66]:
xyz

tensor([[187.1260, 148.2460, 210.4180],
        [185.2560, 152.9680, 204.6180],
        [189.3610, 161.8020, 205.2150],
        [186.0000, 156.5960, 209.9520],
        [181.9480, 158.1870, 213.6110],
        [177.6460, 163.1810, 214.1600],
        [173.8800, 167.3450, 211.8830],
        [166.3370, 171.1260, 210.5930],
        [168.3270, 176.5140, 206.1570],
        [163.8160, 179.3390, 207.0560],
        [160.1640, 182.8460, 207.7550],
        [157.7920, 186.9150, 204.0880],
        [157.1580, 189.0050, 198.6980],
        [152.6600, 192.7810, 192.6450],
        [157.7340, 192.8060, 187.8020],
        [154.6840, 196.1220, 184.1260],
        [153.8000, 200.2520, 181.3890],
        [154.3120, 201.4030, 175.6690],
        [155.5490, 199.9820, 170.5490],
        [154.2110, 199.1620, 162.0550],
        [160.1900, 195.0370, 161.3320],
        [159.2910, 196.5100, 156.3580],
        [161.0400, 197.1880, 150.4170],
        [162.7890, 193.8730, 145.9970],
        [163.7700, 188.6840, 143.8560],


In [70]:
for i in range(len(xyz)):
    if (~torch.isnan(xyz[i])).all():
        break

In [71]:
i

0

In [72]:
xyz = xyz - xyz[i]

In [73]:
xyz

tensor([[  0.0000,   0.0000,   0.0000],
        [ -1.8700,   4.7220,  -5.8000],
        [  2.2350,  13.5560,  -5.2030],
        [ -1.1260,   8.3500,  -0.4660],
        [ -5.1780,   9.9410,   3.1930],
        [ -9.4800,  14.9350,   3.7420],
        [-13.2460,  19.0990,   1.4650],
        [-20.7890,  22.8800,   0.1750],
        [-18.7990,  28.2680,  -4.2610],
        [-23.3100,  31.0930,  -3.3620],
        [-26.9620,  34.6000,  -2.6630],
        [-29.3340,  38.6690,  -6.3300],
        [-29.9680,  40.7590, -11.7200],
        [-34.4660,  44.5350, -17.7730],
        [-29.3920,  44.5600, -22.6160],
        [-32.4420,  47.8760, -26.2920],
        [-33.3260,  52.0060, -29.0290],
        [-32.8140,  53.1570, -34.7490],
        [-31.5770,  51.7360, -39.8690],
        [-32.9150,  50.9160, -48.3630],
        [-26.9360,  46.7910, -49.0860],
        [-27.8350,  48.2640, -54.0600],
        [-26.0860,  48.9420, -60.0010],
        [-24.3370,  45.6270, -64.4210],
        [-23.3560,  40.4380, -66.5620],
